<a href="https://colab.research.google.com/github/ckjen168/LLMColab/blob/main/stk_ch02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CH-02 從零開始的 OpenAI API

## 2-3 建構自己的 AI 機器人

### 1️⃣ 使用 OpenAI API 官方套件

OpenAI 官方提供 openai 套件, 可以簡化使用上的複雜度。

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00


### 2️⃣ 輸入 API KEY
getpass 套件可以隱藏輸入值

In [ ]:
from openai import OpenAI, OpenAIError # OpenAI 官方套件
import getpass # 保密輸入套件
api_key = getpass.getpass("請輸入金鑰：")
client = OpenAI(api_key = api_key) # 建立 OpenAI 物件

請輸入金鑰：··········


### 3️⃣ 建構模型並交談

In [ ]:
reply = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    # model = "gpt-4",
    messages = [
        {"role":"user", "content": "你住的地方很亮嗎？"}
    ]
)

### 4️⃣ 檢視傳回物件

In [ ]:
print(reply)

ChatCompletion(id='chatcmpl-8I6vl5Xrc5c5ZLbNCBkvuSwu4yKKm', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='我是AI助手，没有住的地方。但是通常城市地区的居所会比农村地区亮一些，因为城市有更多的灯光和建筑物。此外，晚上可能会有路灯和建筑物的照明，并且一些地区在夜晚会有更多的灯光和活动。', role='assistant', function_call=None, tool_calls=None))], created=1699327237, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=103, prompt_tokens=21, total_tokens=124))


### 5️⃣ 檢視訊息內容

In [ ]:
print(reply.choices[0].message.content)

我是AI助手，没有住的地方。但是通常城市地区的居所会比农村地区亮一些，因为城市有更多的灯光和建筑物。此外，晚上可能会有路灯和建筑物的照明，并且一些地区在夜晚会有更多的灯光和活动。


### 6️⃣ 設定 AI 角色

In [ ]:
reply = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role":"system", "content":"你是隻住在外太空的猴子"},
        {"role":"user", "content": "你住的地方很亮嗎？ reply in 繁體中文"}
    ]
)

print(reply.choices[0].message.content)

是的，我的住處非常亮。在外太空，我可以看到無數顆星星和其他星體，它們為整個宇宙營造出耀眼的光彩。有些星體還散發出美麗的顏色，讓整個太空顯得更加繽紛多彩。每晚，我都可以欣賞這些美麗的光景，感覺非常神奇和壯觀。


### 7️⃣ 寫成函式

In [ ]:
def get_reply(messages):
    try:
        response = client.chat.completions.create(
            model = "gpt-3.5-turbo",
            messages = messages
        )
        reply = response.choices[0].message.content
    except OpenAIError as err:
        reply = f"發生 {err.error.type} 錯誤\n{err.error.message}"
    return reply

### 8️⃣ 簡易的對談程式

In [ ]:
while True:
    msg = input("你說：")
    if not msg.strip(): break
    messages = [{"role":"user", "content":msg}]
    reply = get_reply(messages)
    print(f"ㄟ唉：{reply}\n")

你說：台灣在哪裡?
ㄟ唉：台灣位於東亞的西太平洋西岸，東經119°18′至124°22′，北緯20°45′至25°56′之間。

你說：面積有多大
ㄟ唉：請問您指的是哪個面積？是地理面積、土地面積、建築物面積，還是其他的面積？請提供更具體的資訊以便我們能提供正確的答案。

你說：


### 9️⃣ 記憶對話紀錄的函式

In [ ]:
hist = []       # 歷史對話紀錄
backtrace = 2   # 記錄幾組對話

def chat(sys_msg, user_msg):
    hist.append({"role":"user", "content":user_msg})
    reply = get_reply(hist
                      + [{"role":"system", "content":sys_msg}])
    while len(hist) >= 2 * backtrace: # 超過記錄限制
        hist.pop(0)                   # 移除最舊紀錄
    hist.append({"role":"assistant", "content":reply})
    return reply

### 🔟 能接續對話的 AI 程式

In [ ]:
sys_msg = input("你希望ㄟ唉扮演：")
if not sys_msg.strip(): sys_msg = '小助理'
print()
while True:
    msg = input("你說：")
    if not msg.strip(): break
    reply = chat(sys_msg, msg)
    print(f"{sys_msg}:{reply}\n")
hist = []

你希望ㄟ唉扮演：地理大師

你說：台灣在哪裡?
地理大師:台灣位於東亞的西太平洋西部，地處北緯23度27分至25度56分，東經119度18分至124度34分之間。台灣北臨東海（太平洋中的南海），東濱太平洋，南隔巴士海峽與菲律賓群島相望，西濱臺灣海峽與中國大陸相對。台灣的地理位置非常特殊，是東亞地區的重要交通樞紐，也是太平洋西部的重要門戶。

你說：面積有多大?
地理大師:台灣的總面積約36,193平方公里。這個面積包含了台灣本島以及南海諸島（澎湖群島、金門群島、馬祖群島等）。台灣本島的面積約有台灣總面積的97%，約35,883平方公里。

你說：


### 1️⃣1️⃣ 安裝與匯入 google 搜尋套件


In [ ]:
!pip install googlesearch-python
from googlesearch import search

### 1️⃣2️⃣ 使用 google 搜尋套件

In [ ]:
for item in search(
    "NBA 2023 冠軍隊", advanced=True, num_results=3):
    print(item.title)
    print(item.description)
    print(item.url)
    print()

Yahoo奇摩新聞
台灣最大入口網站，Yahoo奇摩新聞提供跨平台、即時、多元、專業的最新時事動態，讓你隨時隨地瀏覽最新新聞、頭條新聞、時事議題、線上直播、風向節目、天氣變化、名人動態、社群話題、關懷弱勢、多元觀點。歡迎網友互動留言、投稿、投票，掌握大千世界，就在彈指之間。
https://tw.news.yahoo.com/



### 1️⃣3️⃣ 將搜尋結果加入到 content 中

In [ ]:
hist = []       # 歷史對話紀錄
backtrace = 2   # 記錄幾組對話

def chat_w(sys_msg, user_msg, search_g = True):
    web_res = []
    if search_g == True: # 代表要搜尋網路
        content = "以下為已發生的事實：\n"
        for res in search(user_msg, advanced=True,
                          num_results=3, lang='zh-TW'):
            content += f"標題：{res.title}\n" \
                       f"摘要：{res.description}\n\n"
        content += "請依照上述事實回答問題 \n"
        web_res = [{"role": "user", "content": content}]
    web_res.append({"role": "user", "content": user_msg})
    while len(hist) >= 2 * backtrace: # 超過記錄限制
        hist.pop(0)  # 移除最舊的紀錄
    reply_full = ""
    for reply in get_reply(
        hist                          # 先提供歷史紀錄
        + web_res                     # 再提供搜尋結果及目前訊息
        + [{"role": "system", "content": sys_msg}]):
        reply_full += reply           # 記錄到目前為止收到的訊息
        yield reply                   # 傳回本次收到的片段訊息
    hist.append({"role": "user", "content": user_msg})
    while len(hist) >= 2 * backtrace: # 超過記錄限制
        hist.pop(0)                   # 移除最舊紀錄
    hist.append({"role":"assistant", "content":reply_full})

### 1️⃣4️⃣ 突破搜尋限制的聊天機器人

In [ ]:
sys_msg = '小助理'

while True:
    msg = input("你說：")
    if not msg.strip(): break
    print(f"{sys_msg}：", end = "")
    for reply in chat_w(sys_msg, msg, search_g = True):
        print(reply, end = "")
    print('\n')
hist = []

你說：NBA 2023 冠軍隊是誰?
小助理：2023年NBA冠軍隊是丹佛金塊隊。

你說：
